## Model Example
This notebook demonstrates a model with a recurrent neural network. It uses manual unrolling of the recurrent connections.

In [2]:
import tensorflow as tf
import numpy as np

### Model Class

We can use classes to encapsulate tensorflow models. The below skeleton shows one way of using classes with tensorflow.

The Model class has methods, variables, and properties that capture both the graph and the tensorflow session

#### Tensorflow Graph

A tensorflow graph is a computational graph of different tensorflow operations. It defines the computation and how different operations and tensors relate, but it doesn't actually do the computation or store the values of the variables. All of that magic happens within the tensorflow session.

#### Tensorflow Session

A tensorflow session is the context where values for tensorflow variables are instantiated and computations are run. So if you are saving a model's weights, you are actually saving the weights of the tensorflow session. If you are loading a model's weights, you need to load them into a session. When variables are initialized, that has to happen within a session. In a way, the graph is stateless. State is stored in sessions. The session also takes care of running computations, so if you are running training, those need to be run in the session.

A session is instantiated with a graph, typically the current default graph. A session is only able to run computations on the graph that is tied to the session.

In [3]:
class DeepLearningModel():
    def __init__():
        return
    
    def gen_uniform_random_weights(self, k_out, k_in, scale, dtype=np.float32):
        """
        Returns weights of shape (k_in, k_out) initialized between [-scale, scale]
        """
        return ((np.random.rand(k_in, k_out) * 2 - 1) * scale).astype(dtype)

    def gen_random_weights_tanh(self, k_out, k_in, dtype=np.float32):
        scale = (6. / (k_in + k_out)) ** .5
        return self.gen_uniform_random_weights(k_out, k_in, scale, dtype=dtype)

    def gen_random_weights_sigmoid(self, k_out, k_in, dtype=np.float32):
        scale = 4. * (6. / (k_in + k_out)) ** .5
        return self.gen_uniform_random_weights(k_out, k_in, scale, dtype=dtype)

    def gen_random_weights_reLu(self, k_out, k_in, dtype=np.float32):
        scale = (2. / (k_in + k_out)) ** .5
        return self.gen_uniform_random_weights(k_out, k_in, scale, dtype=dtype)

    def gen_biases(self, k, dtype=np.float32):
        """
        Initialize biases as zero.
        """
        return np.zeros((k, ), dtype=dtype)
    
    def clip_gradient(self, grad, magnitude=1.0):
        """returns a clipped gradient, where it is between [-magnitude and magnitude]"""
        magnitude = abs(magnitude)
        return tf.maximum(tf.minimum(grad, magnitude), - magnitude)

In [28]:
class RecurrentNetworkModel(DeepLearningModel):
    """
    Tutorial Model
    """
    
    def __init__(self, num_layers, k_rnncell, k_input_embedding, number_data_embeddings, time_steps):
        """
        args:
            num_layers: number of hidden layers
            k_hidden: number of units in the hidden layers
            k_input_embedding: dimensionality of the input
            k_softmax: dimensionality of the output layer
        """
        self._graph = None
        self._session = None
        self.num_layers = num_layers
        self.k_rnncell = k_rnncell
        self.k_input_embedding = k_input_embedding
        self.time_steps = time_steps
        self.number_data_embeddings = number_data_embeddings
        
        self._merged_training_summary = None
        self._merged_validation_summary = None
    
    
    def load_model(self, model_filename):
        with self.graph.as_default():
            model_saver = tf.train.Saver()
        
        self._session = tf.Session(graph=self.graph)
        model_saver.restore(self._session, model_filename)
        return
    
    
    def save_model(self, model_filename):
        with self.graph.as_default():
            model_saver = tf.train.Saver()
            
        model_saver.save(self.session, model_filename)
        
    def create_graph(self):
        self.cells = {}
        self.W = {}
        self.b = {}
        self.Z = {}
        self.A = {}
        self.Z_series = {}
        self.A_series = {}
        
        self._graph = tf.Graph()
        
        with self._graph.as_default():
            with tf.name_scope("inputs"):
                self.X = tf.placeholder(tf.int32, shape=(None, self.time_steps))
            
            with tf.name_scope("embeddings"):
                self.data_embeddings = tf.Variable(np.random.rand(self.number_data_embeddings, self.k_input_embedding).astype(np.float32), dtype=tf.float32, name="data_embeddings")
                self.data_embedding_lookup = tf.nn.embedding_lookup(self.data_embeddings, self.X)
                self.A_series[0] = tf.unstack(self.data_embedding_lookup, axis = 1)
            
            with tf.name_scope("recurrent_layers"):
                for layer in range(1, num_layers + 1):
                    if layer == 1:
                        self.W[(layer, layer - 1)] = tf.Variable(self.gen_random_weights_tanh(self.k_rnncell, self.k_input_embedding), dtype=tf.float32, name="W_%i_%i" % (layer, layer - 1))
                    else:
                        self.W[(layer, layer - 1)] = tf.Variable(self.gen_random_weights_tanh(self.k_rnncell, self.k_rnncell), dtype=tf.float32, name="W_%i_%i" % (layer, layer - 1))
                    
                    self.b[layer] = tf.Variable(self.gen_biases(self.k_rnncell), dtype=tf.float32, name="b_%i" % layer)
                    
                    self.Z_series[layer] = []
                    self.A_series[layer] = []
                
                    
                    for X in self.A_series[layer - 1]:
                        Z = tf.add(tf.matmul(X, self.W[(layer, layer - 1)]), self.b[layer])
                        A = tf.nn.tanh(Z)
                        self.Z_series[layer].append(Z)
                        self.A_series[layer].append(A)
                
                    self.Z[layer] = tf.stack(self.Z_series[layer], axis=1)
                    self.A[layer] = tf.stack(self.A_series[layer], axis=1)
                
            self.init_op = tf.global_variables_initializer()
        return
            

    def write_graph(self):
        self.tensorboard_writer.add_graph(self.graph)
        return
    
    def init_model(self, adam_beta1=0.9, adam_beta2=0.999):
        self.session.run(self.init_op)
    
    @property
    def graph(self):
        if self._graph is None:
            self.create_graph()
        return self._graph
    
    @property
    def session(self):
        if self._session is None:
            self._session = tf.Session(graph=self.graph)
        return self._session

In [29]:
num_layers = 2
k_rnncell = 7
k_input_embedding = 12
number_data_embeddings = 100
time_steps = 10

In [30]:
# create a model instance with 2 hidden layers and 10 hidden units.

model_a = RecurrentNetworkModel(num_layers,  k_rnncell, k_input_embedding, number_data_embeddings, time_steps)

In [31]:
model_a.graph

In [32]:
model_a.data_embedding_lookup

<tf.Tensor 'embeddings/embedding_lookup:0' shape=(?, 10, 12) dtype=float32>

In [33]:
model_a.data_embedding_lookup

<tf.Tensor 'embeddings/embedding_lookup:0' shape=(?, 10, 12) dtype=float32>

In [34]:
model_a.init_model()

In [35]:
model_a.data_embedding_lookup.get_shape()

TensorShape([Dimension(None), Dimension(10), Dimension(12)])

In [36]:
model_a.session.run(model_a.data_embeddings)

array([[ 0.7315734 ,  0.15491393,  0.81759852, ...,  0.30380505,
         0.27469048,  0.8195551 ],
       [ 0.58668327,  0.8296172 ,  0.71467352, ...,  0.94868523,
         0.27233857,  0.03905618],
       [ 0.58029145,  0.90169221,  0.69426012, ...,  0.50565505,
         0.20131119,  0.92678499],
       ..., 
       [ 0.53769046,  0.76241189,  0.33025128, ...,  0.72637171,
         0.7556504 ,  0.9180609 ],
       [ 0.18025073,  0.73141003,  0.31048918, ...,  0.74832296,
         0.3530606 ,  0.13091487],
       [ 0.02852038,  0.55394506,  0.9497118 , ...,  0.30654246,
         0.81106067,  0.87170929]], dtype=float32)

In [37]:
data = np.random.randint(0, number_data_embeddings, size=(4, time_steps))

In [38]:
model_a.session.run(model_a.data_embedding_lookup,
                    feed_dict={model_a.X: data}).shape

(4, 10, 12)

In [40]:
model_a.session.run(model_a.Z[1],
                    feed_dict={model_a.X: data}).shape

(4, 10, 7)

In [41]:
model_a.session.run(model_a.Z[2],
                    feed_dict={model_a.X: data}).shape

(4, 10, 7)

In [43]:
%%time
model_a.session.run(model_a.A[2],
                    feed_dict={model_a.X: data}).shape

CPU times: user 1.26 ms, sys: 734 µs, total: 1.99 ms
Wall time: 752 µs


(4, 10, 7)

In [44]:
data = np.random.randint(0, number_data_embeddings, size=(100, time_steps))

In [45]:
%%time
model_a.session.run(model_a.A[2],
                    feed_dict={model_a.X: data}).shape

CPU times: user 1.52 ms, sys: 982 µs, total: 2.5 ms
Wall time: 6.53 ms


(100, 10, 7)

In [46]:
data = np.random.randint(0, number_data_embeddings, size=(1000, time_steps))

In [47]:
%%time
model_a.session.run(model_a.A[2],
                    feed_dict={model_a.X: data}).shape

CPU times: user 4.36 ms, sys: 1.64 ms, total: 6.01 ms
Wall time: 2.03 ms


(1000, 10, 7)

---
The output above shows how to get the rnn_output, it's fairly straightforward to append an output model on the end of the final rnn_states or the rnn_outputs.